In [ ]:
from google.cloud import bigquery
from google.cloud import aiplatform as vertexai
import numpy as np
import pandas as pd

In [1]:
# Retrieve and set PROJECT_ID environment variable
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
print(PROJECT_ID)

york-bb-cohort


In [2]:
# Set REGION environment variable
REGION = 'us-east4'
print(REGION)

us-east4


In [3]:
%%bigquery nested_df --project $PROJECT_ID
SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
TABLESAMPLE SYSTEM (2 PERCENT) 

Query is running:   0%|          |

Downloading:   0%|          |

In [4]:
print(nested_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17637 entries, 0 to 17636
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   visitorId             0 non-null      Int64 
 1   visitNumber           17637 non-null  Int64 
 2   visitId               17637 non-null  Int64 
 3   visitStartTime        17637 non-null  Int64 
 4   date                  17637 non-null  object
 5   totals                17637 non-null  object
 6   trafficSource         17637 non-null  object
 7   device                17637 non-null  object
 8   geoNetwork            17637 non-null  object
 9   customDimensions      17637 non-null  object
 10  hits                  17637 non-null  object
 11  fullVisitorId         17637 non-null  object
 12  userId                0 non-null      object
 13  clientId              0 non-null      object
 14  channelGrouping       17637 non-null  object
 15  socialEngagementType  17637 non-null

In [7]:
cols = ["visitNumber", "visitId", "visitStartTime", "date","fullVisitorId", "channelGrouping", "socialEngagementType"]
for col in cols:
    print(col, nested_df[col].nunique(), nested_df[col].unique())
    

visitNumber 108 <IntegerArray>
[  2,  17,  14,   1,  10,   8,   7,   3,  16,   6,
 ...
  43,  44, 125, 142,  72, 167,  94,  92, 165,  97]
Length: 108, dtype: Int64
visitId 17380 <IntegerArray>
[1486723248, 1486767842, 1486774707, 1486722539, 1486775773, 1486734665,
 1486792302, 1486748004, 1486747995, 1486751942,
 ...
 1474897587, 1474925084, 1474934708, 1474897429, 1474915581, 1474880184,
 1474928938, 1474915297, 1474922295, 1474953356]
Length: 17380, dtype: Int64
visitStartTime 17380 <IntegerArray>
[1486723248, 1486767842, 1486774707, 1486722539, 1486775773, 1486734665,
 1486792302, 1486748004, 1486747995, 1486751942,
 ...
 1474897587, 1474925087, 1474934708, 1474897429, 1474915581, 1474880184,
 1474928938, 1474915297, 1474922295, 1474953356]
Length: 17380, dtype: Int64
date 7 ['20170210' '20161222' '20161217' '20160923' '20161119' '20160816'
 '20160926']
fullVisitorId 16148 ['3537240514615185660' '9334655840242218288' '274075099018939329' ...
 '4137305465781411175' '6966498085786819

In [5]:
%%bigquery --project $PROJECT_ID
SELECT count(visitorId) AS visitorId_not_null_count
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
WHERE visitorId IS NOT NULL

Query is running:   0%|          |

Downloading:   0%|          |

,visitorId_not_null_count
0,0


In [6]:
%%bigquery --project $PROJECT_ID
SELECT count(userId) AS userId_not_null_count
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` 
WHERE userId IS NOT NULL

Query is running:   0%|          |

Downloading:   0%|          |

,userId_not_null_count
0,0


In [8]:
%%bigquery df --project $PROJECT_ID

SELECT * EXCEPT(hits, totals, trafficSource, device, geoNetwork), h.* EXCEPT(product), totals.*, prod.*, trafficSource.*, device.*, geoNetwork.* 
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160801` ,
UNNEST (hits) AS h,
UNNEST(h.product) AS prod


Query is running:   0%|          |

Downloading:   0%|          |

In [9]:
df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,customDimensions,fullVisitorId,userId,channelGrouping,socialEngagementType,...,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,<NA>,1,1470046245,1470046245,20160801,"[{'index': 4, 'value': 'EMEA'}]",895954260133011192,None,Organic Search,Not Socially Engaged,...,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,scalamedia.de,not available in demo dataset,not available in demo dataset,not available in demo dataset
1,<NA>,1,1470046245,1470046245,20160801,"[{'index': 4, 'value': 'EMEA'}]",895954260133011192,None,Organic Search,Not Socially Engaged,...,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,scalamedia.de,not available in demo dataset,not available in demo dataset,not available in demo dataset
2,<NA>,1,1470046245,1470046245,20160801,"[{'index': 4, 'value': 'EMEA'}]",895954260133011192,None,Organic Search,Not Socially Engaged,...,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,scalamedia.de,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,<NA>,1,1470046245,1470046245,20160801,"[{'index': 4, 'value': 'EMEA'}]",895954260133011192,None,Organic Search,Not Socially Engaged,...,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,scalamedia.de,not available in demo dataset,not available in demo dataset,not available in demo dataset
4,<NA>,1,1470046245,1470046245,20160801,"[{'index': 4, 'value': 'EMEA'}]",895954260133011192,None,Organic Search,Not Socially Engaged,...,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,scalamedia.de,not available in demo dataset,not available in demo dataset,not available in demo dataset


In [10]:
df.shape

(93007, 161)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93007 entries, 0 to 93006
Columns: 161 entries, visitorId to networkLocation
dtypes: Int64(41), boolean(15), object(105)
memory usage: 109.9+ MB


In [12]:
df.isnull().sum()

visitorId          93007
visitNumber            0
visitId                0
visitStartTime         0
date                   0
                   ...  
cityId                 0
networkDomain          0
latitude               0
longitude              0
networkLocation        0
Length: 161, dtype: int64

Looks like visitorId and userId has all null values.

In [14]:
%%bigquery df --project $PROJECT_ID

SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`

Query is running:   0%|          |

Downloading:   0%|          |

In [15]:
%%bigquery totals_df --project $PROJECT_ID

SELECT totals.* FROM  `bigquery-public-data.google_analytics_sample.ga_sessions_*`

Query is running:   0%|          |

Downloading:   0%|          |

In [16]:
# Fields of total 
print(totals_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype
---  ------                   --------------   -----
 0   visits                   903653 non-null  Int64
 1   hits                     903653 non-null  Int64
 2   pageviews                903553 non-null  Int64
 3   timeOnSite               451894 non-null  Int64
 4   bounces                  450630 non-null  Int64
 5   transactions             11552 non-null   Int64
 6   transactionRevenue       11515 non-null   Int64
 7   newVisits                703060 non-null  Int64
 8   screenviews              0 non-null       Int64
 9   uniqueScreenviews        0 non-null       Int64
 10  timeOnScreen             0 non-null       Int64
 11  totalTransactionRevenue  11515 non-null   Int64
 12  sessionQualityDim        68379 non-null   Int64
dtypes: Int64(13)
memory usage: 100.8 MB
None


In [17]:
totals_df.nunique()

visits                        1
hits                        274
pageviews                   213
timeOnSite                 4260
bounces                       1
transactions                 13
transactionRevenue         5332
newVisits                     1
screenviews                   0
uniqueScreenviews             0
timeOnScreen                  0
totalTransactionRevenue    6200
sessionQualityDim            99
dtype: int64

In [18]:
cols = ["visits", "bounces", "newVisits", "transactions"]
for col in cols:
    print(col, totals_df[col].unique())
    

visits <IntegerArray>
[1]
Length: 1, dtype: Int64
bounces <IntegerArray>
[1, <NA>]
Length: 2, dtype: Int64
newVisits <IntegerArray>
[1, <NA>]
Length: 2, dtype: Int64
transactions <IntegerArray>
[<NA>, 1, 2, 4, 3, 5, 6, 8, 10, 21, 25, 7, 12, 15]
Length: 14, dtype: Int64


In [19]:
%%bigquery device_df --project $PROJECT_ID

SELECT device.* FROM  `bigquery-public-data.google_analytics_sample.ga_sessions_*`

Query is running:   0%|          |

Downloading:   0%|          |

In [20]:
print(device_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   browser                    903653 non-null  object 
 1   browserVersion             903653 non-null  object 
 2   browserSize                903653 non-null  object 
 3   operatingSystem            903653 non-null  object 
 4   operatingSystemVersion     903653 non-null  object 
 5   isMobile                   903653 non-null  boolean
 6   mobileDeviceBranding       903653 non-null  object 
 7   mobileDeviceModel          903653 non-null  object 
 8   mobileInputSelector        903653 non-null  object 
 9   mobileDeviceInfo           903653 non-null  object 
 10  mobileDeviceMarketingName  903653 non-null  object 
 11  flashVersion               903653 non-null  object 
 12  javaEnabled                0 non-null       boolean
 13  language                   90

In [21]:
device_df.nunique()

browser                      54
browserVersion                1
browserSize                   1
operatingSystem              20
operatingSystemVersion        1
isMobile                      2
mobileDeviceBranding          1
mobileDeviceModel             1
mobileInputSelector           1
mobileDeviceInfo              1
mobileDeviceMarketingName     1
flashVersion                  1
javaEnabled                   0
language                      1
screenColors                  1
screenResolution              1
deviceCategory                3
dtype: int64

In [22]:
cols = device_df.columns
for col in cols:
    print(col, device_df[col].unique())

browser ['BlackBerry' 'Firefox' 'Safari' 'Nintendo Browser' 'Edge'
 'Internet Explorer' 'Opera Mini' 'UC Browser' 'DoCoMo'
 'Mozilla Compatible Agent' 'Chrome' 'NokiaE52-1' 'Safari (in-app)'
 'Puffin' '(not set)' 'YaBrowser' 'Reddit' 'Lunascape'
 'osee2unifiedRelease' 'Opera' 'SeaMonkey' 'Konqueror' 'Seznam' 'Mozilla'
 'Amazon Silk' 'Android Webview' 'Android Browser' 'Coc Coc'
 'Nokia Browser' 'Hisense M20-M_LTE' 'TCL P500M' 'HTC802t_TD' 'YE'
 'Android Runtime' 'M5' 'MRCHROME' 'Maxthon' 'Iron' 'IE with Chrome Frame'
 'Nichrome' 'LYF_LS_4002_12' 'DASH_JR_3G'
 '[Use default User-agent string] LIVRENPOCHE' 'ADM' 'Apple-iPhone7C2'
 'User Agent' 'LYF_LS_4002_11' '0' 'subjectAgent: NoticiasBoom'
 'MQQBrowser' 'no-ua' 'ThumbSniper' 'Changa 99695759' 'CSM Click']
browserVersion ['not available in demo dataset']
browserSize ['not available in demo dataset']
operatingSystem ['BlackBerry' 'Firefox OS' 'Windows Phone' 'Nintendo Wii' 'Xbox' 'Samsung'
 'Nintendo WiiU' 'FreeBSD' 'NTT DoCoMo' 'SunOS'

In [23]:
%%bigquery geoNetwork_df --project $PROJECT_ID

SELECT geoNetwork.* FROM  `bigquery-public-data.google_analytics_sample.ga_sessions_*`

Query is running:   0%|          |

Downloading:   0%|          |

In [24]:
geoNetwork_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   continent        903653 non-null  object
 1   subContinent     903653 non-null  object
 2   country          903653 non-null  object
 3   region           903653 non-null  object
 4   metro            903653 non-null  object
 5   city             903653 non-null  object
 6   cityId           903653 non-null  object
 7   networkDomain    903653 non-null  object
 8   latitude         903653 non-null  object
 9   longitude        903653 non-null  object
 10  networkLocation  903653 non-null  object
dtypes: object(11)
memory usage: 75.8+ MB


In [25]:
geoNetwork_df.nunique()

continent              6
subContinent          23
country              222
region               376
metro                 94
city                 649
cityId                 1
networkDomain      28064
latitude               1
longitude              1
networkLocation        1
dtype: int64

In [26]:
geoNetwork_df.drop("networkDomain", axis=1, inplace=True)
cols = geoNetwork_df.columns
for col in cols:
    print(col, geoNetwork_df[col].unique())

continent ['Africa' 'Americas' 'Asia' 'Oceania' 'Europe' '(not set)']
subContinent ['Northern Africa' 'South America' 'Western Asia' 'Australasia'
 'Eastern Europe' 'Caribbean' 'Middle Africa' 'Southern Asia'
 'Southern Europe' 'Eastern Asia' 'Western Europe' 'Southern Africa'
 'Eastern Africa' 'Northern Europe' 'Central America' 'Western Africa'
 'Southeast Asia' '(not set)' 'Central Asia' 'Micronesian Region'
 'Northern America' 'Melanesia' 'Polynesia']
country ['Egypt' 'Ecuador' 'Bahrain' 'New Zealand' 'Armenia' 'Bulgaria' 'Haiti'
 'Czechia' 'Angola' 'Pakistan' 'Croatia' 'Greece' 'China' 'Belgium'
 'Portugal' 'Slovakia' 'Austria' 'Switzerland' 'South Africa' 'Sri Lanka'
 'Bangladesh' 'Morocco' 'Kenya' 'Hungary' 'Chile' 'Serbia' 'Lithuania'
 'Tunisia' 'United Arab Emirates' 'Dominican Republic' 'Algeria'
 'Azerbaijan' 'Costa Rica' 'Uruguay' 'Saudi Arabia' 'Bolivia' 'Jordan'
 'Palestine' 'Central African Republic' 'Norway' 'Yemen' 'Guinea'
 'Nigeria' 'Uganda' 'Albania' 'Slovenia' 'Bru

In [38]:
%%bigquery trafficSource_df --project $PROJECT_ID

SELECT trafficSource.* FROM  `bigquery-public-data.google_analytics_sample.ga_sessions_*`

Query is running:   0%|          |

Downloading:   0%|          |

In [39]:
print(trafficSource_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   referralPath      330941 non-null  object 
 1   campaign          903653 non-null  object 
 2   source            903653 non-null  object 
 3   medium            903653 non-null  object 
 4   keyword           241905 non-null  object 
 5   adContent         10946 non-null   object 
 6   adwordsClickInfo  903653 non-null  object 
 7   isTrueDirect      274005 non-null  boolean
 8   campaignCode      1 non-null       object 
dtypes: boolean(1), object(8)
memory usage: 56.9+ MB
None


In [28]:
%%bigquery customDim_df --project $PROJECT_ID

SELECT cd.* FROM  `bigquery-public-data.google_analytics_sample.ga_sessions_*`, UNNEST (customDimensions) AS cd

Query is running:   0%|          |

Downloading:   0%|          |

In [29]:
print(customDim_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711673 entries, 0 to 711672
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   index   711673 non-null  Int64 
 1   value   711673 non-null  object
dtypes: Int64(1), object(1)
memory usage: 11.5+ MB
None


In [30]:
customDim_df.nunique()

index    1
value    5
dtype: int64

In [31]:
for col in customDim_df.columns:
    print(col, customDim_df[col].unique())

index <IntegerArray>
[4]
Length: 1, dtype: Int64
value ['North America' 'South America' 'EMEA' 'APAC' 'Central America']


In [32]:
%%bigquery hits_df --project $PROJECT_ID

SELECT h.* FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`, UNNEST(hits) AS h

Query is running:   0%|          |

Downloading:   0%|          |

In [33]:
hits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13006 entries, 0 to 13005
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   hitNumber            13006 non-null  Int64  
 1   time                 13006 non-null  Int64  
 2   hour                 13006 non-null  Int64  
 3   minute               13006 non-null  Int64  
 4   isSecure             0 non-null      boolean
 5   isInteraction        13006 non-null  boolean
 6   isEntrance           1711 non-null   boolean
 7   isExit               1711 non-null   boolean
 8   referer              1686 non-null   object 
 9   page                 13006 non-null  object 
 10  transaction          5786 non-null   object 
 11  item                 5786 non-null   object 
 12  contentInfo          0 non-null      object 
 13  appInfo              13006 non-null  object 
 14  exceptionInfo        13006 non-null  object 
 15  eventInfo            2988 non-null  

In [34]:
%%bigquery hits_exp_df --project $PROJECT_ID

SELECT h.time, h.eCommerceAction.*, h.page.pagepath, h.page.pageTitle, 
p.productSKU, p.v2ProductCategory, p.productPrice
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`, 
UNNEST(hits) AS h, UNNEST(product) AS p

Query is running:   0%|          |

Downloading:   0%|          |

In [35]:
#pd.json_normalize(hits_exp_df, record_path=['eCommerceAction'])
hits_exp_df = hits_exp_df.infer_objects()
hits_exp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93007 entries, 0 to 93006
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   time               93007 non-null  Int64 
 1   action_type        93007 non-null  object
 2   step               93007 non-null  Int64 
 3   option             0 non-null      object
 4   pagepath           93007 non-null  object
 5   pageTitle          93007 non-null  object
 6   productSKU         93007 non-null  object
 7   v2ProductCategory  93007 non-null  object
 8   productPrice       93007 non-null  Int64 
dtypes: Int64(3), object(6)
memory usage: 6.7+ MB


In [36]:
hits_exp_df.describe()

,time,step,productPrice
count,93007.0,93007.0,93007.0
mean,325540.159472,1.0,23281332.104035
std,690355.822998,0.0,25731794.115954
min,0.0,1.0,0.0
25%,33872.0,1.0,7990000.0
50%,111426.0,1.0,16990000.0
75%,294843.0,1.0,21990000.0
max,7507769.0,1.0,575700000.0


In [37]:
hits_exp_df.nunique()

time                 8268
action_type             7
step                    1
option                  0
pagepath              208
pageTitle             170
productSKU            361
v2ProductCategory      36
productPrice          156
dtype: int64